In [0]:
# Install required libraries
!pip install azure-storage-file-datalake

In [0]:
from azure.storage.filedatalake import DataLakeServiceClient
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Constants
ACCOUNT_NAME = "cdmo"
ACCOUNT_KEY = "XXXXXXXXXXXX"  # Replace with your actual key or use environment variables
CONTAINERS = ["00-landing", "01-bronze", "02-silver", "03-gold", "config"]
SOURCES = ["customerfeedback", "manufacturebatch", "productformula", "sales", "supplier"]

# Initialize Azure Data Lake Service Client
def initialize_adls_client(account_name, account_key):
    try:
        service_client = DataLakeServiceClient(
            account_url=f"https://{account_name}.dfs.core.windows.net",
            credential=account_key,
        )
        logger.info("Successfully initialized Azure Data Lake Service Client.")
        return service_client
    except Exception as e:
        logger.error(f"Failed to initialize Azure Data Lake Service Client: {e}")
        raise

# Create a container in ADLS
def create_container(service_client, container_name):
    try:
        container_client = service_client.get_file_system_client(container_name)
        if not container_client.exists():
            container_client.create_file_system()
            logger.info(f"Container '{container_name}' created successfully.")
        else:
            logger.warning(f"Container '{container_name}' already exists.")
    except Exception as e:
        logger.error(f"Error creating container '{container_name}': {e}")
        raise

# Create a folder in a container
def create_folder(service_client, container_name, folder_path):
    try:
        file_system_client = service_client.get_file_system_client(container_name)
        directory_client = file_system_client.get_directory_client(folder_path)
        if not directory_client.exists():
            directory_client.create_directory()
            logger.info(f"Folder '{folder_path}' created successfully in container '{container_name}'.")
        else:
            logger.warning(f"Folder '{folder_path}' already exists in container '{container_name}'.")
    except Exception as e:
        logger.error(f"Error creating folder '{folder_path}' in container '{container_name}': {e}")
        raise

# Main function to set up ADLS structure
def setup_adls_structure(service_client, containers, sources):
    try:
        # Step 1: Create required containers
        for container in containers:
            create_container(service_client, container)

        # Step 2: Create folder structures in the '00-landing' container
        for source in sources:
            # Create folders in the 'incoming' directory
            incoming_folder_path = f"data/incoming/{source}/"
            create_folder(service_client, "00-landing", incoming_folder_path)

            # Create folders in the 'archive' directory
            archive_folder_path = f"data/archive/{source}/"
            create_folder(service_client, "00-landing", archive_folder_path)

        logger.info("ADLS structure setup completed successfully.")
    except Exception as e:
        logger.error(f"Error setting up ADLS structure: {e}")
        raise

# Execute the setup
if __name__ == "__main__":
    try:
        service_client = initialize_adls_client(ACCOUNT_NAME, ACCOUNT_KEY)
        setup_adls_structure(service_client, CONTAINERS, SOURCES)
    except Exception as e:
        logger.error(f"Script execution failed: {e}")